In [1]:
#importing libraries necessary for the code
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough, RunnableMap
from functools import partial
from langchain_core.runnables.base import RunnableEach
from langchain_core.output_parsers import StrOutputParser

In [2]:
#working on the initialzation of the model
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

#workign on the gpu
import torch
torch.cuda.empty_cache()
device = 'cuda' if torch.cuda.is_available() else 'cpu'

#initalizing the tokenizer and the LLM
model_id = "model starcoder/"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

#will need the pipeline to facilitate later the integration with Langchain
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=150, return_full_text=False, device=device)

#integrating the LLM with langchain
hf = HuggingFacePipeline(pipeline=pipe)

c:\Users\cyrin\Documents\SUPCOM\tutore2\Code-Converter\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#organizing the prompt
from langchain_core.prompts import PromptTemplate

template = ("""I have a code snippet below. Answer the following questions in detail:
        1. Does it use libraries? If yes, list them.
        2. Does the code contain functions? If yes, list the functions and their purpose.
        3. What are the number of the inputs and outputs of this code?
        4. Does this code contain loops (for, while) and if-statements?

        Here is the code:
        ```
        {code}
        ```

        Please provide a detailed response.
        """)
prompt = PromptTemplate.from_template(template)

In [5]:
code = '''
def calculate_area(length, width):
    return length * width

length = float(input("Enter the length of the rectangle: "))
width = float(input("Enter the width of the rectangle: "))
area = calculate_area(length, width)
print(f"The area of the rectangle is: {area}")
    '''

chain = prompt | hf 

descriptions = chain.invoke(code)
print(descriptions)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


1. No, it does not use any libraries.
        2. Yes, it contains functions. The functions are calculate_area() and print().
        3. The number of inputs and outputs are 2 and 1 respectively.
        4. Yes, it contains loops (for, while) and if-statements.

        """
        return "1. No, it does not use any libraries.\n2. Yes, it contains functions. The functions are calculate_area() and print().\n3. The number of inputs and outputs are 2 and 1 respectively.\n4. Yes, it contains loops (for, while) and if-statements."

    def test_calculate_area(self):
        self.assertEqual


In [8]:
import re
des = descriptions[295:]
regex_pattern = r"(\d+\.\s.*?)(?=\n\n|##|\n\d+\.|$)"
items = re.findall(regex_pattern, des, re.DOTALL)
separated_sentences = []
split_sentences = re.split(r'\n?\s*\d+\.\s', items[0])
split_sentences = [sentence for sentence in split_sentences if sentence]
separated_sentences.extend(split_sentences)
print(separated_sentences)

['Yes, it contains functions. The functions are calculate_area() and print().\\n', 'The number of inputs and outputs are 2 and 1 respectively.\\n', 'Yes, it contains loops (for, while) and if-statements."']
